<a href="https://colab.research.google.com/github/maddataanalyst/gamebook_tools/blob/main/gamebook_tester/gamebook_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by Filip Wójcik
[Official website](https://filip-wojcik.com)

# Intro

This notebook contains a simple script to run **gamebooks** written in **twine** and exported to HTML.

What does the gamebook player do?

1. Open a file with the game;
2. Click selectors (**tw-link**);
3. Run until no options are available.

# Setup

First, we need to setup Selenium and Chrome browser on Colab.

**If you want to use this on local computer - you need "normal" Chrome and Selenium installation for your operating system!!!***

In [1]:
%pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 27.3 MB/s eta 0:00:00


# Gamebook tester

In order to run it:

1. Put the `.html` file into the "files" directory.
2. Run the code below - it collectes "paths" several times.
3. Check results - analyze freuency of results per character, etc., etc.

In [2]:
# Google Colab specific code - installs and updates Selenium + Chrome on Collab.
# If you want to use it locally: follow instructions for your operating system and Python setup for Chrome + Selenium.
import google_colab_selenium as gs

driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import time
import os
import random
import pandas as pd
from tqdm.notebook import tqdm
from selenium.webdriver.common.by import By
from IPython.display import display, clear_output

**SET PATH TO GAME FILE (html) HERE**


In [4]:
GAMEBOOK_FILE_PATH  = "testgame.html"

**SET NUMBER OF GAME PLAYS** (number of "paths" to collect)

In [5]:
gameplays = 5
if gameplays < 0:
    raise ValueError("This needs to be above 0")

## Test game

Run the cell below

In [6]:
game_full_path = os.path.abspath(os.path.join(os.curdir, GAMEBOOK_FILE_PATH))

def check_single_path():
    path = []

    # New game - new driver. This is not nice, I need to find a workaround.
    # This is caused by Twine games (written in Harlowe) DO NOT REFRESH after page reload
    driver = gs.Chrome()
    driver.get(f"file://{game_full_path}")
    while True:
        links = driver.find_elements(By.TAG_NAME, "tw-link")
        valid_links = [l for l in links if l.text is not None and l.text != '']
        if not valid_links:
            break

        random_link = random.choice(valid_links)
        path.append(random_link.text)
        random_link.click()
        time.sleep(0.03)
    time.sleep(0.1)
    driver.quit()
    return path

paths = []
for i in tqdm(range(gameplays)):
    clear_output(wait=True)
    display(i)
    paths.append(check_single_path())


4

<IPython.core.display.Javascript object>

## Analyze results

Now you can analyze results, depending on your game

In [7]:
paths

[['haker', 'first_mission', 'second_mission', 'haker_wins'],
 ['soldier', 'first_mission', 'second_mission', 'soldier_wins'],
 ['detective', 'first_mission', 'second_mission', 'detective_wins'],
 ['haker', 'first_mission', 'second_mission', 'haker_wins'],
 ['detective', 'first_mission', 'second_mission', 'detective_wins']]

In [8]:
characters_selected = [pth[0] for pth in paths]
pd.value_counts(characters_selected)

<ipython-input-8-07d1308c2a8d>:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(characters_selected)
<ipython-input-8-07d1308c2a8d>:2: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts(characters_selected)


,count
haker,2
detective,2
soldier,1


In [9]:
endings = [pth[-1] for pth in paths]
pd.value_counts(endings)

<ipython-input-9-3c51c58bb3b7>:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(endings)
<ipython-input-9-3c51c58bb3b7>:2: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts(endings)


,count
haker_wins,2
detective_wins,2
soldier_wins,1
